In [252]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./레이블합친최종본v2.csv',encoding='cp949')
df = df[['회사명','거래소코드','회계년도','당좌자산구성비율','타인자본회전율','매출채권회전율','순운전자본비율','차입금의존도','총자본투자효율',
         '재고자산_대_유동자산비율','총자본영업이익율','자기자본순이익율','CASH_FLOW_대_매출액비율','매출원가감소율여부','배당금감소율여부',
         '변화율여부','직원감소율여부','자본증가율여부','투자활동감소율여부','회생기업2']]
df['회계년도'] = pd.to_datetime(df['회계년도'], format='20%d-%m').dt.strftime('20%d%m')
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)
df['거래소코드'] = df['거래소코드'].astype(int)
df = df.sort_values(by='회계년도')
df = df.reset_index(drop=True)
df['회계년도'] = df['회계년도'].astype(int)
df

,회사명,거래소코드,회계년도,당좌자산구성비율,타인자본회전율,매출채권회전율,순운전자본비율,차입금의존도,총자본투자효율,재고자산_대_유동자산비율,총자본영업이익율,자기자본순이익율,CASH_FLOW_대_매출액비율,매출원가감소율여부,배당금감소율여부,변화율여부,직원감소율여부,자본증가율여부,투자활동감소율여부,회생기업2
0,CMG제약,58820,201112,46.14,2.01,2.29,28.23,7.32,12.72,17.88,2.37,-8.28,-24.07,1,0,0,0,0,0,0
1,쌍방울,102280,201112,27.11,2.92,6.00,30.40,24.67,17.75,59.50,6.41,8.59,5.86,0,0,0,0,0,1,0
2,파라텍,33540,201112,53.68,3.85,4.26,36.25,4.65,56.51,9.89,7.59,8.64,4.62,0,0,0,0,0,0,0
3,듀오백,73190,201112,51.67,6.13,11.52,44.89,4.08,17.07,9.25,2.04,4.49,6.94,0,0,0,0,0,1,0
4,웨이브일렉트로닉스,95270,201112,49.11,2.58,4.90,37.24,15.42,0.49,19.09,-15.21,-22.52,13.93,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8780,비덴트,121800,201912,10.59,0.32,1.15,-2.72,1.84,-1.60,26.20,-0.60,-3.02,-6.48,0,0,0,0,0,1,0
8781,범양건영,2410,201912,50.62,4.04,9.75,22.30,11.67,21.03,1.08,8.92,32.82,5.79,0,0,0,0,0,0,0
8782,유성티엔에스,24800,201912,31.49,1.89,7.15,-10.48,25.07,17.95,1.34,1.95,11.23,4.13,0,0,0,0,0,0,0
8783,대산업,3310,201912,16.57,2.29,7.09,3.33,18.03,14.26,40.70,3.06,3.51,2.35,0,0,0,0,0,0,0


In [253]:
# medi = lambda x: x.fillna(x.mean())
# df = df.groupby('회사명').apply(medi)
# df = df.dropna()
# df.isna().sum()

### 피쳐와 레이블 나누기

In [254]:
data = df.iloc[:,1:19]
target = df.iloc[:,19]

In [255]:
data

,거래소코드,회계년도,당좌자산구성비율,타인자본회전율,매출채권회전율,순운전자본비율,차입금의존도,총자본투자효율,재고자산_대_유동자산비율,총자본영업이익율,자기자본순이익율,CASH_FLOW_대_매출액비율,매출원가감소율여부,배당금감소율여부,변화율여부,직원감소율여부,자본증가율여부,투자활동감소율여부
0,58820,201112,46.14,2.01,2.29,28.23,7.32,12.72,17.88,2.37,-8.28,-24.07,1,0,0,0,0,0
1,102280,201112,27.11,2.92,6.00,30.40,24.67,17.75,59.50,6.41,8.59,5.86,0,0,0,0,0,1
2,33540,201112,53.68,3.85,4.26,36.25,4.65,56.51,9.89,7.59,8.64,4.62,0,0,0,0,0,0
3,73190,201112,51.67,6.13,11.52,44.89,4.08,17.07,9.25,2.04,4.49,6.94,0,0,0,0,0,1
4,95270,201112,49.11,2.58,4.90,37.24,15.42,0.49,19.09,-15.21,-22.52,13.93,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8780,121800,201912,10.59,0.32,1.15,-2.72,1.84,-1.60,26.20,-0.60,-3.02,-6.48,0,0,0,0,0,1
8781,2410,201912,50.62,4.04,9.75,22.30,11.67,21.03,1.08,8.92,32.82,5.79,0,0,0,0,0,0
8782,24800,201912,31.49,1.89,7.15,-10.48,25.07,17.95,1.34,1.95,11.23,4.13,0,0,0,0,0,0
8783,3310,201912,16.57,2.29,7.09,3.33,18.03,14.26,40.70,3.06,3.51,2.35,0,0,0,0,0,0


In [256]:
target

0       0
1       0
2       0
3       0
4       0
       ..
8780    0
8781    0
8782    0
8783    0
8784    0
Name: 회생기업2, Length: 8785, dtype: int64

In [257]:
data.columns

Index(['거래소코드', '회계년도', '당좌자산구성비율', '타인자본회전율', '매출채권회전율', '순운전자본비율', '차입금의존도',
       '총자본투자효율', '재고자산_대_유동자산비율', '총자본영업이익율', '자기자본순이익율', 'CASH_FLOW_대_매출액비율',
       '매출원가감소율여부', '배당금감소율여부', '변화율여부', '직원감소율여부', '자본증가율여부', '투자활동감소율여부'],
      dtype='object')

### 스케일링을 위해 데이터를 나눠줍니다

In [258]:
df_feature = data.iloc[:,2:12] # 스케일링을 사용할 데이터
df_front = data.iloc[:,0:2] # 거래소코드와 회계년도는 스케일링은 하면 안되니 따로 뺀다
df_semi = data.iloc[:,12:] # 반정형 역시 따로 뺀다

### 데이터 분리

In [259]:
# 기간을 나누기전에 스케일링 부터 합시당!
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
ss_sc = SS.fit_transform(df_feature)
ss_sc2 = pd.DataFrame(ss_sc)
ss_sc2.columns=['당좌자산구성비율', '타인자본회전율', '매출채권회전율', '순운전자본비율', '차입금의존도',
       '총자본투자효율', '재고자산_대_유동자산비율', '총자본영업이익율', '자기자본순이익율', 'CASH_FLOW_대_매출액비율']
ss_sc2 = pd.concat([df_front,ss_sc2],axis=1)
ss_sc2 = pd.concat([ss_sc2,df_semi],axis=1)

# 기간을 얼마나 나누게 될까요~?
x_train = ss_sc2.loc[:5641] # 2011 ~ 2016년
x_test = ss_sc2.loc[5642:] # 2017 ~ 2019년
y_train = target.loc[:5641] # 2011 ~ 2016년
y_test = target.loc[5642:] # 2017 ~ 2019년

In [260]:
print('X 테스트 데이터 :')
print(x_test)
print('y 테스트 데이터 :')
print(list(y_test))

X 테스트 데이터 :
       거래소코드    회계년도  당좌자산구성비율   타인자본회전율   매출채권회전율   순운전자본비율    차입금의존도  \
5642  208860  201712 -0.094405 -0.412310 -0.206651  0.329705 -0.988045   
5643   67390  201712 -0.832572 -0.823868 -0.235554 -0.032705  1.025821   
5644     700  201712  0.279214 -0.142668 -0.075434  0.248420  0.097090   
5645   19010  201712 -1.154599 -0.937401  0.805514 -1.117158  1.022790   
5646   46940  201712 -0.731088 -0.180513  0.030927  0.206390  0.660270   
...      ...     ...       ...       ...       ...       ...       ...   
8780  121800  201912 -1.541824 -0.951593 -0.311856 -0.669896 -0.876500   
8781    2410  201912  0.727670  0.808173  0.185267  0.322171 -0.280585   
8782   24800  201912 -0.356902 -0.208896  0.034974 -0.977587  0.531751   
8783    3310  201912 -1.202789 -0.019674  0.031505 -0.430007  0.104971   
8784    3280  201912 -1.009460 -1.017821 -0.213010 -1.560456  0.124977   

       총자본투자효율  재고자산_대_유동자산비율  총자본영업이익율  자기자본순이익율  CASH_FLOW_대_매출액비율  \
5642 -0.660614      -1.3832

### 오버샘플링을 해봅시다

In [239]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(x_train,y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (5642, 18) (5642,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (10934, 18) (10934,)
SMOTE 적용 후 레이블 값 분포: 
 0    5467
1    5467
Name: 회생기업2, dtype: int64


### 언더샘플링을 해봅시다

In [261]:
from imblearn.under_sampling import RandomUnderSampler

undo = RandomUnderSampler()
x_train_under, y_train_under = undo.fit_resample(x_train, y_train)
print('Under Sampling 적용 전 학습용 피처/레이블 데이터 세트: ', x_train.shape, y_train.shape)
print('Under Sampling 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('Under Sampling 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

Under Sampling 적용 전 학습용 피처/레이블 데이터 세트:  (5642, 18) (5642,)
Under Sampling 적용 후 학습용 피처/레이블 데이터 세트:  (10934, 18) (10934,)
Under Sampling 적용 후 레이블 값 분포: 
 0    5467
1    5467
Name: 회생기업2, dtype: int64


In [262]:
# 피처, 레이블 데이터 분포 비율

train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(x_train.shape, x_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/train_cnt)

학습 세트 Shape:(5642, 18), 테스트 세트 Shape:(3143, 18)
 학습 세트 레이블 값 분포 비율
0    0.968983
1    0.031017
Name: 회생기업2, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.547678
1    0.009394
Name: 회생기업2, dtype: float64


### 로지스틱 회귀분석

In [263]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import TimeSeriesSplit 

lg = LogisticRegression()
param_grid = [{'C':np.linspace(0.1,10,100)}]
gs = GridSearchCV(estimator=lg, param_grid=param_grid, scoring='accuracy',cv=TimeSeriesSplit(n_splits=5),n_jobs=1)
gs.fit(x_train_under,y_train_under)
best_est = gs.best_estimator_
predictions = best_est.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print('GridSearchCV 최적 하이퍼 파라미터 :', gs.best_params_)
print('GridSearchCV 최고 정확도 :{0:.4f}'.format(gs.best_score_))
print('테스트 세트에서의 LogisticRegression 정확도 : {0:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 : {'C': 0.1}
GridSearchCV 최고 정확도 :nan
테스트 세트에서의 LogisticRegression 정확도 : 0.5068


In [264]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, f1: {3:.4f}'.format(accuracy, precision, recall,f1))
    
get_clf_eval(y_test, predictions)

오차 행렬
[[1571 1519]
 [  31   22]]
정확도: 0.5068, 정밀도: 0.0143, 재현율: 0.4151, f1: 0.0276


### 랜덤 포레스트

In [266]:
# 랜덤포레스트 + 그리드서치로 모델 학습
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=1)
param_grid_rf = [{'n_estimators': range(5,50,100), 'max_depth': range(3,5), 'max_features': range(1,4)}]
rf_gs = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='accuracy', cv=TimeSeriesSplit(n_splits=5), n_jobs=1)
rf_gs.fit(x_train_under,y_train_under)
best_est = rf_gs.best_estimator_
predictions = best_est.predict(x_test)
accuracy = accuracy_score(y_test, predictions)
print('GridSearchCV 최적 하이퍼 파라미터 :', rf_gs.best_params_)
print('GridSearchCV 최고 정확도 :{0:.4f}'.format(rf_gs.best_score_))
print('테스트 세트에서의 LogisticRegression 정확도 : {0:.4f}'.format(accuracy))

GridSearchCV 최적 하이퍼 파라미터 : {'max_depth': 3, 'max_features': 1, 'n_estimators': 5}
GridSearchCV 최고 정확도 :0.5586
테스트 세트에서의 LogisticRegression 정확도 : 0.4703


In [267]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, f1: {3:.4f}'.format(accuracy, precision, recall,f1))
    
get_clf_eval(y_test, predictions)

오차 행렬
[[1432 1658]
 [   7   46]]
정확도: 0.4703, 정밀도: 0.0270, 재현율: 0.8679, f1: 0.0524


### XGBoost

In [270]:
#필요한 라이브러리 임포트
from xgboost import XGBClassifier

#XGBoost 정의
#파라미터는 이전 글을 참고하여 grid 설정(dict 형태)
xgb= XGBClassifier()

xgb_param_grid={
    'n_estimators' : [100,200,300,400,500],
    'learning_rate' : [0.01,0.05,0.1,0.15],
    'max_depth' : [3,5,7,10,15],
    'gamma' : [0,1,2,3],
    'colsample_bytree' : [0.8,0.9],
    
}

#score종류는 acc,f1,f1_micro,f1_macro등 원하는걸로 설정)
#여기서 설정 파라미터의 갯수(총 4000개의 조합이므로 4000번의 학습이 돌아감)
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="f1_weighted", n_jobs=-1, verbose = 2,cv=TimeSeriesSplit(n_splits=5))
xgb_grid.fit(x_train_under, y_train_under)

#best f1_macro 수치와 best parameter확인
print("best f1_weighted : {0: .4f}".format(xgb_grid.best_score_))
print("best param : ",xgb_grid.best_params_)

#dataframe으로 랭킹순보기
result_df = pd.DataFrame(xgb_grid.cv_results_)
result_df.sort_values(by=['rank_test_score'],inplace=True)

#plot
result_df[['params','mean_test_score','rank_test_score']].head(10)

Fitting 5 folds for each of 800 candidates, totalling 4000 fits
best f1_weighted :  0.4457
best param :  {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 500}


,params,mean_test_score,rank_test_score
484,"{'colsample_bytree': 0.9, 'gamma': 0, 'learnin...",0.445676,1
578,"{'colsample_bytree': 0.9, 'gamma': 1, 'learnin...",0.444699,2
579,"{'colsample_bytree': 0.9, 'gamma': 1, 'learnin...",0.444699,2
0,"{'colsample_bytree': 0.8, 'gamma': 0, 'learnin...",0.443327,4
374,"{'colsample_bytree': 0.8, 'gamma': 3, 'learnin...",0.443327,4
373,"{'colsample_bytree': 0.8, 'gamma': 3, 'learnin...",0.443327,4
372,"{'colsample_bytree': 0.8, 'gamma': 3, 'learnin...",0.443327,4
371,"{'colsample_bytree': 0.8, 'gamma': 3, 'learnin...",0.443327,4
370,"{'colsample_bytree': 0.8, 'gamma': 3, 'learnin...",0.443327,4
369,"{'colsample_bytree': 0.8, 'gamma': 3, 'learnin...",0.443327,4


In [271]:
import xgboost as xgb
from xgboost import XGBClassifier

xgb_wrapper = XGBClassifier(n_estimators=500, learning_rate=0.15, max_depth=5,colsmple_pytree=0.9)
xgb_wrapper.fit(x_train_under, y_train_under)
w_preds = xgb_wrapper.predict(x_test)
w_pred_proba = xgb_wrapper.predict(x_test)

get_clf_eval(y_test, w_preds)

[21:50:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsmple_pytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


오차 행렬
[[2049 1041]
 [  27   26]]
정확도: 0.6602, 정밀도: 0.0244, 재현율: 0.4906, f1: 0.0464


### LGBM

In [268]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier()
start = time.time()
parameters = {'num_leaves':[20,40,60,80,100], 'min_child_samples':[5,10,15],'max_depth':[-1,5,10,20],
             'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}#Define the scoring

clf=GridSearchCV(lgb,parameters,scoring='accuracy',cv=TimeSeriesSplit(n_splits=5))
clf.fit(X=x_train_under, y=y_train_under)

print(clf.best_params_)
predicted=clf.predict(x_test)
print('Classification of the result is:')
print(accuracy_score(y_test, predicted))
end = time.time()
print('Execution time is:')
print(end - start)

{'learning_rate': 0.1, 'max_depth': 5, 'min_child_samples': 5, 'num_leaves': 20, 'reg_alpha': 0}
Classification of the result is:
0.6401527203308941
Execution time is:
55.17732548713684


In [269]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier()
start = time.time()

lgb_cr = LGBMClassifier(learning_rate=0.1, max_depth=5,min_child_samples=5,num_leaves=20,reg_alpha=0)
lgb_cr.fit(x_train_under, y_train_under)
w_preds = lgb_cr.predict(x_test)
w_pred_proba = lgb_cr.predict(x_test)

get_clf_eval(y_test, w_preds)

오차 행렬
[[1984 1106]
 [  25   28]]
정확도: 0.6402, 정밀도: 0.0247, 재현율: 0.5283, f1: 0.0472
